In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

# Create a Model class that inherits nn.Module

class Model(nn.Module):
  # input layer(4 features of the flower)-->
  # Hidden Layer1 (number of neurons) -->
  # Hidden Layer2(number of neurons) -->
  # output(3 classes of iris flower)
  def __init__(self, in_features=4, h1=8, h2=9, out_features=3):
    super().__init__() #instantiate our nn.Module
    self.fc1 = nn.Linear(in_features, h1) # fully connected 1(fc1)
    self.fc2 = nn.Linear(h1, h2)
    self.out = nn.Linear(h2, out_features)

  def forward(self,x):
    x = F.relu(self.fc1(x)) # Rectified linear unit
    x = F.relu(self.fc2(x))
    x = self.out(x)

    return x
# Pick a manual seed for randomization
torch.manual_seed(41)
# create an instance of model
model = Model()

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
%matplotlib inline

url = 'https://gist.githubusercontent.com/curran/a08a1080b88344b0c8a7/raw/0e7a9b0a5d22642a06d3d5b9bcbad9890c8ee534/iris.csv'
my_df = pd.read_csv(url)
# change last column from strings to integers
my_df['species'] = my_df['species'].replace({'setosa':0, 'versicolor':1, 'virginica':2})
my_df # gets the first 5 data


In [ ]:
# Train Test Split! Set X,y
X = my_df.drop('species', axis=1).values # convert these to numpy arrays using the values
y = my_df['species'].values


In [ ]:
from sklearn.model_selection import train_test_split

# Train Test Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=41) # test size is 20%

#Convert X features to float tensors
X_train = torch.FloatTensor(X_train)
X_test = torch.FloatTensor(X_test)

# convert y labels to tensors long
y_train = torch.LongTensor(y_train)
y_test = torch.LongTensor(y_test)

# Set the criterion of model to measure the error,
# how far off the predictions are from the data
criterion = nn.CrossEntropyLoss()
# choose Adam Optimizer, learning rate(if error doesn`t go down after a bunch of iterations(epochs), lower the learning rate)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

In [ ]:
# Train our model!
# Epochs?(one run through all the training data in our networks)
epochs = 100
losses = []
for i in range(epochs):
  # Go forward and get a prediction
  y_pred = model.forward(X_train) # Get predicted results

  # Measure the loss/error, gonna be high at first
  loss = criterion(y_pred, y_train) # compare predicted values and trained values
  # keep track of the losses
  losses.append(loss.detach().numpy())

  #print every 10 epoch
  if i % 10 == 0:
    print(f'Epoch: {i} Loss: {loss}')

  # Do some back propagation: take the error rate of forward propagation and feed it back
  # thru the network to fine tune the weights
  optimizer.zero_grad()
  loss.backward()
  optimizer.step()


In [ ]:
# graph it up
plt.plot(range(epochs), losses)
plt.ylabel('Loss')
plt.xlabel('Epoch')

In [ ]:
# Evaluate Model on Test Data Set (validate model on test set)
with torch.no_grad(): # Turn off back propagation
   y_eval = model.forward(X_test) # X_test are features from our test set, y_eval will be predictions
   loss = criterion(y_eval, y_test) # Find the loss


In [ ]:
correct = 0
with torch.no_grad():
  for i, data in enumerate(X_test):
    y_val = model.forward(data)

    # will tell us what type of flower class our network thinks it is
    print(f'{i+1:2}. {str(y_val):38} {y_test[i]} {y_val.argmax().item()}') # the highest number is what the flower thinks it is

    # correct or not
    if y_val.argmax().item() == y_test[i]:
      correct +=1

print(f'\n{correct} out of {len(y_test)}')

In [ ]:
new_iris = torch.tensor([5.6, 3.2, 4.1, 2.1]) # new data points

with torch.no_grad():
  print(model(new_iris))


In [ ]:
# Save our NN Model

torch.save(model.state_dict(), 'iris_model.pt')

# Load the Saved Model
new_model = Model()
new_model.load_state_dict(torch.load('iris_model.pt')) # Loading the weights and bias that are saved
# Make sure it is loaded correctly
model.eval()
